In [1]:
import re, subprocess, boto3, json, shlex, mysql, os, urllib, logging
import pandas as pd
import numpy as np
from s3path import S3Path
from pathlib import Path
from tqdm.notebook import tqdm
from packaging import version
pd.set_option("display.max_colwidth", 40)

In [2]:
from utils.utils import *

In [3]:
import glob

# Functions

## List FASTQs

In [4]:
# FASTQ reads/indices required for each workflow
fastq_map = {
    'genomic': ['R2'],
    'barcode': ['R1']
}

# Get fastq file paths on S3 for each file id
# Returns dictionary from id to s3 path
# Throws exception if FASTQs don't exist for any id
def get_fastqs(
    path: str, # path to directory containing FASTQ files
    fastq_file_ids: list, # FASTQ file ids needed for this run type (e.g. I1, R1, R2, etc.)

):
    fastq_map = dict()
    _, bucket, key, _, _ = urllib.parse.urlsplit(path)
    for fid in fastq_file_ids:
        files = get_s3_objects(
            bucket, key.lstrip("/"),
            re.compile(f"_{fid}_\d{{3}}.fastq.gz$")
        )
        try:
            assert files, f"AssertionError: Missing `{fid}` archives!"
            fastq_map[fid] = [os.path.join("s3://", bucket, str(f)) for f in files]
        except AssertionError as err:
            logging.warning("%s\n\t %s", err, path)
            return
    return fastq_map

In [5]:
def get_barcode_genomic_fastqs(samples):
    
    samples['fastqBarcode'] = np.empty((len(samples), 0)).tolist()
    samples['fastqGenomic'] = np.empty((len(samples), 0)).tolist()
    
    for sample, row in samples.iterrows():
        S3_path = row['S3_path']
        
        # Barcode FASTQ files
        fastq_file_ids = fastq_map['barcode']        
        barcode_path = f"{S3_path}/barcode/"
        fastqBarcode = get_fastqs(barcode_path, fastq_file_ids)['R1']
        
        # Genomic FASTQ files
        fastq_file_ids = fastq_map['genomic']        
        genomic_path = f"{S3_path}/genomic/"
        fastqGenomic = get_fastqs(genomic_path, fastq_file_ids)['R2']
        
        samples.loc[sample, 'fastqBarcode'] += fastqBarcode
        samples.loc[sample, 'fastqGenomic'] += fastqGenomic
    
    return samples

## DB queries

In [6]:
# Common query col: id, request_id, Sample
def get_sample_name(query, query_col, creds):
    
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.sample_data"
        query = f"""
        SELECT {table_sample_data}.Sample
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        sample_names = []
        results = execute_query(query, user, password)
        for result in results:
            sample_names.append(result[0])
        return sample_names
    except Error as e:
        print(f"Error: {e}")
    
    
def get_aws_path(query, query_col, creds):
    
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.sample_data"
        query = f"""
        SELECT {table_sample_data}.AWS_storage
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        sample_paths = []
        results = execute_query(query, user, password)
        for result in results:
            sample_paths.append(result[0])
        return sample_paths
    except Error as e:
        print(f"Error: {e}")
        
            
def get_sample_id(query, query_col, creds):
    
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.sample_data"
        query = f"""
        SELECT {table_sample_data}.id
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        sample_ids = []
        results = execute_query(query, user, password)
        for result in results:
            sample_ids.append(result[0])
        return sample_ids
    except Error as e:
        print(f"Error: {e}")
        
def format_sample_aws(querys, query_col, creds):
    sample_names = []
    sample_paths = []
    sample_ids = []
    
    for query in querys:
        sample_names += get_sample_name(query, query_col, creds)
        sample_paths += get_aws_path(query, query_col, creds)
        sample_ids += get_sample_id(query, query_col, creds)
        
    sample_paths = [s.strip('/') for s in sample_paths] # remove trailing slash if exists
    
    samples = pd.DataFrame(
        dict(S3_path=sample_paths, Sample_ID=sample_ids),
        index=sample_names,
        dtype=str,
    )
    return samples

In [7]:
def get_genomeIndex_id(query, query_col, creds):
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.sample_data"
        query = f"""
        SELECT {table_sample_data}.genomeIndex_id
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        
        results = execute_query(query, user, password)
        for result in results:
            genomeIndex_id = result[0]
            
        return genomeIndex_id
            
    except Error as e:
        print(f"Error: {e}")
        
def get_scTech_id(query, query_col, creds):
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.genome_index"
        query = f"""
        SELECT {table_sample_data}.scTech_id
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        
        results = execute_query(query, user, password)
        for result in results:
            scTech_id = result[0]
            
        return scTech_id
            
    except Error as e:
        print(f"Error: {e}")
        
def get_index(query, query_col, creds):
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.genome_index"
        query = f"""
        SELECT {table_sample_data}.gIndex
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        
        results = execute_query(query, user, password)
        for result in results:
            index = result[0]
            
        return index
            
    except Error as e:
        print(f"Error: {e}")
        
def get_assay(query, query_col, creds):
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.sc_tech"
        query = f"""
        SELECT {table_sample_data}.Run_name
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        
        results = execute_query(query, user, password)
        for result in results:
            assay = result[0]
            
        return assay
            
    except Error as e:
        print(f"Error: {e}")
        
def get_barcode(query, query_col, creds):
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.sc_tech"
        query = f"""
        SELECT {table_sample_data}.barcodes
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        
        results = execute_query(query, user, password)
        for result in results:
            barcode = result[0]
            
        return barcode
            
    except Error as e:
        print(f"Error: {e}")
        
        
def format_assay_barcode(samples, creds):
    
    # samples['genomeIndex_id'] = np.nan
    samples['index'] = np.nan
    # samples['scTech_id'] = np.nan
    samples['assay'] = np.nan
    samples['barcode'] = np.nan
    
    for sample, row in samples.iterrows():
        sample_id = row['Sample_ID']
        
        genomeIndex_id  = get_genomeIndex_id(sample_id, 'id', creds)
        index = get_index(genomeIndex_id, 'id', creds)
        scTech_id = get_scTech_id(genomeIndex_id, 'id', creds)
        assay = get_assay(scTech_id, 'id', creds)
        barcode = get_barcode(scTech_id, 'id', creds)
        
        # samples.loc[sample, 'genomeIndex_id'] = genomeIndex_id
        samples.loc[sample, 'index'] = index
        # samples.loc[sample, 'scTech_id'] = scTech_id
        samples.loc[sample, 'assay'] = assay
        samples.loc[sample, 'barcode'] = barcode

    return samples

In [8]:
def get_project_id(sample_id, creds):
    
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.sample_data"
        table_project_data = "peer_lab_db.project_data"
        query = f"""
        SELECT {table_project_data}.projectName
        FROM {table_project_data}
        LEFT JOIN {table_sample_data}
        ON {table_project_data}.id = {table_sample_data}.projectData_id
        WHERE {table_sample_data}.id = {sample_id}
        """
        result = execute_query(query, user, password)[0][0]
        return result
    except Error as e:
        print(f"Error: {e}")

# Process Samples

## Setup

In [9]:
# Maps from .wdl name (prefix) to results dirname
results_dirs = {
    "SeqcAda": "seqc-ada-results",
}

# Maps from .wdl name (prefix) to shell script
sh_files = {
    "SeqcAda": "submit.sh",
}

In [10]:
# Location of docker files
common_docker_registry = "quay.io/hisplan"

prefix = "SeqcAda" # Workflow to run; also .wdl filename prefix
pipeline_type = prefix # field in *.labels.json
output_dirname = results_dirs[prefix]

# If need to add comment, put here
email = '20noor.sohail@gmail.com'
comment = "sohailn"

In [11]:
# Locations of workflow-related directories and files
path_to_cromwell_secrets = f"{Path.home()}/.cromwell/cromwell-secrets.json" # CHANGE THIS
workflow_dir = glob.glob(f"{Path.home()}/scing/bin/*ada*")[0]
path_to_exec = sh_files[prefix]
config_dir = f"{workflow_dir}/configs"
path_to_options = glob.glob(f"{workflow_dir}/*.options.aws.json")[0]

# Other file locations
db_credentials_path = f"{Path.home()}/.config.json" # CHANGE THIS

In [12]:
# Set credentials based on SCRIdb CLI config file
with open(db_credentials_path) as f:
    creds = json.load(f)

# Excution

## Sample information

In [15]:
# Common query col: id, request_id, Sample
request_ids = ['ARN-1167']

samples = format_sample_aws(request_ids, 'request_id', creds)
samples.head()

,S3_path,Sample_ID
ARN-1167_M-4T1,s3://dp-lab-data/collaborators/aboir...,3249
ARN-1167_Normal,s3://dp-lab-data/collaborators/aboir...,3250
ARN-1167_PM-4T1,s3://dp-lab-data/collaborators/aboir...,3251


In [16]:
samples = format_assay_barcode(samples, creds)
samples.head()

,S3_path,Sample_ID,index,assay,barcode
ARN-1167_M-4T1,s3://dp-lab-data/collaborators/aboir...,3249,s3://seqc-public/genomes/mm38_long_p...,ten_x_v3,s3://seqc-public/barcodes/ten_x_v3/f...
ARN-1167_Normal,s3://dp-lab-data/collaborators/aboir...,3250,s3://seqc-public/genomes/mm38_long_p...,ten_x_v3,s3://seqc-public/barcodes/ten_x_v3/f...
ARN-1167_PM-4T1,s3://dp-lab-data/collaborators/aboir...,3251,s3://seqc-public/genomes/mm38_long_p...,ten_x_v3,s3://seqc-public/barcodes/ten_x_v3/f...


In [17]:
samples = get_barcode_genomic_fastqs(samples)
samples.head()

,S3_path,Sample_ID,index,assay,barcode,fastqBarcode,fastqGenomic
ARN-1167_M-4T1,s3://dp-lab-data/collaborators/aboir...,3249,s3://seqc-public/genomes/mm38_long_p...,ten_x_v3,s3://seqc-public/barcodes/ten_x_v3/f...,[s3://dp-lab-data/collaborators/aboi...,[s3://dp-lab-data/collaborators/aboi...
ARN-1167_Normal,s3://dp-lab-data/collaborators/aboir...,3250,s3://seqc-public/genomes/mm38_long_p...,ten_x_v3,s3://seqc-public/barcodes/ten_x_v3/f...,[s3://dp-lab-data/collaborators/aboi...,[s3://dp-lab-data/collaborators/aboi...
ARN-1167_PM-4T1,s3://dp-lab-data/collaborators/aboir...,3251,s3://seqc-public/genomes/mm38_long_p...,ten_x_v3,s3://seqc-public/barcodes/ten_x_v3/f...,[s3://dp-lab-data/collaborators/aboi...,[s3://dp-lab-data/collaborators/aboi...


## Make input file

In [18]:
template_prefix = 'template'
# Load minimum inputs and labels fields from input template
with open(f"{config_dir}/{template_prefix}.inputs.json") as f:
    std_inputs_fields = list(json.load(f).keys())
    
# Annotate all samples with workflow inputs and labels
inputs = pd.DataFrame(index=samples.index, columns=std_inputs_fields,)
inputs

,SeqcAda.version,SeqcAda.assay,SeqcAda.index,SeqcAda.barcodeFiles,SeqcAda.fastqBarcode,SeqcAda.fastqGenomic,SeqcAda.filterMode,SeqcAda.outputPrefix,SeqcAda.starArguments,SeqcAda.email,SeqcAda.dockerRegistry
ARN-1167_M-4T1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ARN-1167_Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ARN-1167_PM-4T1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Default inputs
inputs[f"{prefix}.version"] = "0.2.11"
inputs[f"{prefix}.filterMode"] = "scRNA-seq"
inputs[f"{prefix}.starArguments"] = "runRNGseed=0"
inputs[f"{prefix}.email"] = email
inputs[f"{prefix}.dockerRegistry"] = common_docker_registry

inputs

,SeqcAda.version,SeqcAda.assay,SeqcAda.index,SeqcAda.barcodeFiles,SeqcAda.fastqBarcode,SeqcAda.fastqGenomic,SeqcAda.filterMode,SeqcAda.outputPrefix,SeqcAda.starArguments,SeqcAda.email,SeqcAda.dockerRegistry
ARN-1167_M-4T1,0.2.11,NaN,NaN,NaN,NaN,NaN,scRNA-seq,NaN,runRNGseed=0,20noor.sohail@gmail.com,quay.io/hisplan
ARN-1167_Normal,0.2.11,NaN,NaN,NaN,NaN,NaN,scRNA-seq,NaN,runRNGseed=0,20noor.sohail@gmail.com,quay.io/hisplan
ARN-1167_PM-4T1,0.2.11,NaN,NaN,NaN,NaN,NaN,scRNA-seq,NaN,runRNGseed=0,20noor.sohail@gmail.com,quay.io/hisplan


In [20]:
# Sample inputs
inputs[f"{prefix}.assay"] = samples["assay"]
inputs[f"{prefix}.index"] = samples["index"]
inputs[f"{prefix}.barcodeFiles"] = samples["barcode"]
inputs[f"{prefix}.fastqBarcode"] = samples["fastqBarcode"]
inputs[f"{prefix}.fastqGenomic"] = samples["fastqGenomic"]

inputs[f"{prefix}.outputPrefix"] = samples.index.tolist()

inputs

,SeqcAda.version,SeqcAda.assay,SeqcAda.index,SeqcAda.barcodeFiles,SeqcAda.fastqBarcode,SeqcAda.fastqGenomic,SeqcAda.filterMode,SeqcAda.outputPrefix,SeqcAda.starArguments,SeqcAda.email,SeqcAda.dockerRegistry
ARN-1167_M-4T1,0.2.11,ten_x_v3,s3://seqc-public/genomes/mm38_long_p...,s3://seqc-public/barcodes/ten_x_v3/f...,[s3://dp-lab-data/collaborators/aboi...,[s3://dp-lab-data/collaborators/aboi...,scRNA-seq,ARN-1167_M-4T1,runRNGseed=0,20noor.sohail@gmail.com,quay.io/hisplan
ARN-1167_Normal,0.2.11,ten_x_v3,s3://seqc-public/genomes/mm38_long_p...,s3://seqc-public/barcodes/ten_x_v3/f...,[s3://dp-lab-data/collaborators/aboi...,[s3://dp-lab-data/collaborators/aboi...,scRNA-seq,ARN-1167_Normal,runRNGseed=0,20noor.sohail@gmail.com,quay.io/hisplan
ARN-1167_PM-4T1,0.2.11,ten_x_v3,s3://seqc-public/genomes/mm38_long_p...,s3://seqc-public/barcodes/ten_x_v3/f...,[s3://dp-lab-data/collaborators/aboi...,[s3://dp-lab-data/collaborators/aboi...,scRNA-seq,ARN-1167_PM-4T1,runRNGseed=0,20noor.sohail@gmail.com,quay.io/hisplan


In [21]:
# Additional changes
inputs[f"{prefix}.index"] = "s3://dp-lab-data/collaborators/aboire/LeptomeningealMetHeterogeneity/transgene_reference/refdata-seqc/STAR-index/"
inputs

,SeqcAda.version,SeqcAda.assay,SeqcAda.index,SeqcAda.barcodeFiles,SeqcAda.fastqBarcode,SeqcAda.fastqGenomic,SeqcAda.filterMode,SeqcAda.outputPrefix,SeqcAda.starArguments,SeqcAda.email,SeqcAda.dockerRegistry
ARN-1167_M-4T1,0.2.11,ten_x_v3,s3://dp-lab-data/collaborators/aboir...,s3://seqc-public/barcodes/ten_x_v3/f...,[s3://dp-lab-data/collaborators/aboi...,[s3://dp-lab-data/collaborators/aboi...,scRNA-seq,ARN-1167_M-4T1,runRNGseed=0,20noor.sohail@gmail.com,quay.io/hisplan
ARN-1167_Normal,0.2.11,ten_x_v3,s3://dp-lab-data/collaborators/aboir...,s3://seqc-public/barcodes/ten_x_v3/f...,[s3://dp-lab-data/collaborators/aboi...,[s3://dp-lab-data/collaborators/aboi...,scRNA-seq,ARN-1167_Normal,runRNGseed=0,20noor.sohail@gmail.com,quay.io/hisplan
ARN-1167_PM-4T1,0.2.11,ten_x_v3,s3://dp-lab-data/collaborators/aboir...,s3://seqc-public/barcodes/ten_x_v3/f...,[s3://dp-lab-data/collaborators/aboi...,[s3://dp-lab-data/collaborators/aboi...,scRNA-seq,ARN-1167_PM-4T1,runRNGseed=0,20noor.sohail@gmail.com,quay.io/hisplan


## Make label file

In [22]:
# Load minimum inputs and labels fields from labels template
with open(f"{config_dir}/{template_prefix}.labels.json") as f:
    std_labels_fields = list(json.load(f).keys())
    
# Annotate all samples with workflow labels
labels = pd.DataFrame(index=samples.index, columns=std_labels_fields,)
labels

,pipelineType,project,sample,owner,destination,transfer,comment
ARN-1167_M-4T1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ARN-1167_Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ARN-1167_PM-4T1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Annotate labels
labels["pipelineType"] = pipeline_type
labels["project"] = samples["Sample_ID"].apply(lambda x: get_project_id(x, creds))
labels["sample"] = labels.index
labels["owner"] = creds["user"]
labels["destination"] = samples['S3_path'] + "/" + output_dirname
labels["transfer"] = "-"
labels["comment"] = creds["user"]

labels

,pipelineType,project,sample,owner,destination,transfer,comment
ARN-1167_M-4T1,SeqcAda,Leptomeningeal metastasis heterogeneity,ARN-1167_M-4T1,sohailn,s3://dp-lab-data/collaborators/aboir...,-,sohailn
ARN-1167_Normal,SeqcAda,Leptomeningeal metastasis heterogeneity,ARN-1167_Normal,sohailn,s3://dp-lab-data/collaborators/aboir...,-,sohailn
ARN-1167_PM-4T1,SeqcAda,Leptomeningeal metastasis heterogeneity,ARN-1167_PM-4T1,sohailn,s3://dp-lab-data/collaborators/aboir...,-,sohailn


# Submit job

In [24]:
inputs

,SeqcAda.version,SeqcAda.assay,SeqcAda.index,SeqcAda.barcodeFiles,SeqcAda.fastqBarcode,SeqcAda.fastqGenomic,SeqcAda.filterMode,SeqcAda.outputPrefix,SeqcAda.starArguments,SeqcAda.email,SeqcAda.dockerRegistry
ARN-1167_M-4T1,0.2.11,ten_x_v3,s3://dp-lab-data/collaborators/aboir...,s3://seqc-public/barcodes/ten_x_v3/f...,[s3://dp-lab-data/collaborators/aboi...,[s3://dp-lab-data/collaborators/aboi...,scRNA-seq,ARN-1167_M-4T1,runRNGseed=0,20noor.sohail@gmail.com,quay.io/hisplan
ARN-1167_Normal,0.2.11,ten_x_v3,s3://dp-lab-data/collaborators/aboir...,s3://seqc-public/barcodes/ten_x_v3/f...,[s3://dp-lab-data/collaborators/aboi...,[s3://dp-lab-data/collaborators/aboi...,scRNA-seq,ARN-1167_Normal,runRNGseed=0,20noor.sohail@gmail.com,quay.io/hisplan
ARN-1167_PM-4T1,0.2.11,ten_x_v3,s3://dp-lab-data/collaborators/aboir...,s3://seqc-public/barcodes/ten_x_v3/f...,[s3://dp-lab-data/collaborators/aboi...,[s3://dp-lab-data/collaborators/aboi...,scRNA-seq,ARN-1167_PM-4T1,runRNGseed=0,20noor.sohail@gmail.com,quay.io/hisplan


In [25]:
labels

,pipelineType,project,sample,owner,destination,transfer,comment
ARN-1167_M-4T1,SeqcAda,Leptomeningeal metastasis heterogeneity,ARN-1167_M-4T1,sohailn,s3://dp-lab-data/collaborators/aboir...,-,sohailn
ARN-1167_Normal,SeqcAda,Leptomeningeal metastasis heterogeneity,ARN-1167_Normal,sohailn,s3://dp-lab-data/collaborators/aboir...,-,sohailn
ARN-1167_PM-4T1,SeqcAda,Leptomeningeal metastasis heterogeneity,ARN-1167_PM-4T1,sohailn,s3://dp-lab-data/collaborators/aboir...,-,sohailn


In [26]:
stdouts = [] # to store all outputs
process = True

with tqdm(samples.index) as t:

    for sample_name in t:

        # Write inputs and labels to file
        path_to_inputs = f"{config_dir}/{sample_name}_{prefix}.inputs.json"
        with open(path_to_inputs, "w") as f_inputs:
            json.dump(inputs.loc[sample_name].to_dict(), f_inputs, indent=4, cls=NpEncoder)

        path_to_labels = f"{config_dir}/{sample_name}_{prefix}.labels.json"
        with open(path_to_labels, "w") as f_labels:
            json.dump(labels.loc[sample_name].to_dict(), f_labels, indent=4, cls=NpEncoder)

        if process:
            stdouts.append(run(
                workflow_path = workflow_dir,
                execp = path_to_exec,
                secrets = path_to_cromwell_secrets,
                inputs = path_to_inputs,
                labels = path_to_labels,
                options = path_to_options,
            ))

  0%|          | 0/3 [00:00<?, ?it/s]

In [27]:
print(path_to_inputs)
print(path_to_labels)

/Users/sohailn/scing/bin/seqc-ada-0.0.4/configs/ARN-1167_PM-4T1_SeqcAda.inputs.json
/Users/sohailn/scing/bin/seqc-ada-0.0.4/configs/ARN-1167_PM-4T1_SeqcAda.labels.json


In [28]:
stdouts

[{'args': ['/Users/sohailn/scing/bin/seqc-ada-0.0.4/submit.sh',
   '-k',
   '/Users/sohailn/.cromwell/cromwell-secrets.json',
   '-i',
   '/Users/sohailn/scing/bin/seqc-ada-0.0.4/configs/ARN-1167_M-4T1_SeqcAda.inputs.json',
   '-l',
   '/Users/sohailn/scing/bin/seqc-ada-0.0.4/configs/ARN-1167_M-4T1_SeqcAda.labels.json',
   '-o',
   '/Users/sohailn/scing/bin/seqc-ada-0.0.4/SeqcAda.options.aws.json'],
  'returncode': 0,
  'stdout': '{"id":"b29e38bf-8da9-4dd2-b1d6-68421dd00357","status":"Submitted"}\n',
  'stderr': ''},
 {'args': ['/Users/sohailn/scing/bin/seqc-ada-0.0.4/submit.sh',
   '-k',
   '/Users/sohailn/.cromwell/cromwell-secrets.json',
   '-i',
   '/Users/sohailn/scing/bin/seqc-ada-0.0.4/configs/ARN-1167_Normal_SeqcAda.inputs.json',
   '-l',
   '/Users/sohailn/scing/bin/seqc-ada-0.0.4/configs/ARN-1167_Normal_SeqcAda.labels.json',
   '-o',
   '/Users/sohailn/scing/bin/seqc-ada-0.0.4/SeqcAda.options.aws.json'],
  'returncode': 0,
  'stdout': '{"id":"870495d0-1c22-44bb-ab21-51ad8e465